<h1><center>CS 455/595a: MLP Demo using TensorFlow</center></h1>
<center>Richard S. Stansbury</center>

This notebook applies the ANN techniques for the Titanic Survivors and Boston Housing Prediction models covered in [1] with the [Titanic](https://www.kaggle.com/c/titanic/) and [Boston Housing](https://scikit-learn.org/stable/modules/generated/sklearn.datasets.load_boston.html) data sets for DT-based classification and regression, respectively.

Several different approaches to model construction are shown ihe demos below

Reference:

[1] Aurelen Geron. *Hands on Machine Learning with Scikit-Learn & TensorFlow* O'Reilley Media Inc, 2017.

[2] Aurelen Geron. "ageron/handson-ml: A series of Jupyter notebooks that walk you through the fundamentals of Machine Learning and Deep Learning in python using Scikit-Learn and TensorFlow." Github.com, online at: https://github.com/ageron/handson-ml [last accessed 2019-03-01]

**Table of Contents**
1. [Titanic Survivor ANN Classifiers](#Titanic-Survivor-Classifier)
 
2. [Boston Housing Cost Ensemble ANN Regressor](#Boston-Housing-Cost-Estimator)

# Titanic Survivor Classifier

## Set up - Imports of libraries and Data Preparation

In [1]:
from matplotlib import pyplot as plt
%matplotlib inline 
import numpy as np
import pandas as pd
import os

# From: https://github.com/ageron/handson-ml/blob/master/09_up_and_running_with_tensorflow.ipynb    
def reset_graph():
    tf.reset_default_graph() 
    
from datetime import datetime

now = datetime.utcnow().strftime("%Y%m%d%H%M%S")
root_logdir = "titanic-logs"
logdir = "{}/run-{}/".format(root_logdir, now)

Import the data and apply pipelines to pre-process the data.

In [2]:
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.base import TransformerMixin, BaseEstimator
from sklearn.model_selection import train_test_split

# Read data from input files into Pandas data frames
data_path = os.path.join("datasets","titanic")
train_filename = "train.csv"
test_filename = "test.csv"

def read_csv(data_path, filename):
    joined_path = os.path.join(data_path, filename)
    return pd.read_csv(joined_path)

# Read CSV file into Pandas Dataframes
train_df = read_csv(data_path, train_filename)

# Defining Data Pre-Processing Pipelines
class DataFrameSelector(BaseEstimator, TransformerMixin):
    
    def __init__(self, attributes):
        self.attributes = attributes
    
    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        return X[self.attributes]

class MostFrequentImputer(BaseEstimator, TransformerMixin):
    
    def fit(self, X, y=None):
        self.most_frequent = pd.Series([X[c].value_counts().index[0] for c in X], 
                                       index = X.columns)
        return self
    
    def transform(self, X):
        return X.fillna(self.most_frequent)


numeric_pipe = Pipeline([
        ("Select", DataFrameSelector(["Age", "Fare", "SibSp", "Parch"])), # Selects Fields from dataframe
        ("Imputer", SimpleImputer(strategy="median")),   # Fills in NaN w/ median value for its column
        ("Scaler", StandardScaler()),
    ])

categories_pipe = Pipeline([
        ("Select", DataFrameSelector(["Pclass", "Sex"])), # Selects Fields from dataframe
        ("MostFreqImp", MostFrequentImputer()), # Fill in NaN with most frequent
        ("OneHot", OneHotEncoder(sparse=False, categories='auto')), # Onehot encode
    ])

preprocessing_pipe = FeatureUnion(transformer_list = [
        ("numeric pipeline", numeric_pipe), 
        ("categories pipeline", categories_pipe)
     ]) 

# Process Input Data Using Pipleines
X_data = preprocessing_pipe.fit_transform(train_df)
y_data = train_df["Survived"].values.reshape(-1,1)

# Process the output data.
feature_names = ["Age", "Fare", "SibSp", "Parch", "Class0", "class1","Sex0", "Sex1"]

print(X_data.shape)
print(y_data.shape)

(891, 9)
(891, 1)


Split the data into a training and validation set.

In [3]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_data, y_data, test_size = 0.33)
print(X_train.shape, y_train.shape, X_test.shape, y_test.shape)

(596, 9) (596, 1) (295, 9) (295, 1)


Implementation of the TF.Estimator.DNNClassifier (formerly of TFLearn)

In [4]:
# Construction Phase

import tensorflow as tf

reset_graph()

feature_cols = [tf.feature_column.numeric_column("X", shape=[X_data.shape[1]])]

dnn_clf = tf.estimator.DNNClassifier(hidden_units=[20,20], n_classes=2,
                                     feature_columns=feature_cols)

train_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={"X": X_train}, y=y_train, batch_size=50, num_epochs=400, shuffle=True)
dnn_clf.train(input_fn=train_input_fn)

test_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={"X": X_test}, y=y_test, shuffle=False)
eval_results = dnn_clf.evaluate(input_fn=test_input_fn)
                                
eval_results


INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': 'C:\\Users\\richa\\AppData\\Local\\Temp\\tmp4m0cs18a', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x0000025CECCB3898>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
To 

INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2019-04-08T13:15:56Z
INFO:tensorflow:Graph was finalized.
Instructions for updating:
Use standard file APIs to check for files with this prefix.
INFO:tensorflow:Restoring parameters from C:\Users\richa\AppData\Local\Temp\tmp4m0cs18a\model.ckpt-4768
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2019-04-08-13:15:57
INFO:tensorflow:Saving dict for global step 4768: accuracy = 0.77627116, accuracy_baseline = 0.61355937, auc = 0.8430988, auc_precision_recall = 0.8034408, average_loss = 0.53733635, global_step = 4768, label/mean = 0.38644066, loss = 52.838074, precision = 0.7790698, prediction/mean = 0.3744611, recall = 0.5877193
INFO:tensorflow:Saving 'checkpoint_path' summary for global step 4768: C:\Users\richa\AppData\Local\Temp\tmp4m0cs18a\model.ckpt-4768


{'accuracy': 0.77627116,
 'accuracy_baseline': 0.61355937,
 'auc': 0.8430988,
 'auc_precision_recall': 0.8034408,
 'average_loss': 0.53733635,
 'label/mean': 0.38644066,
 'loss': 52.838074,
 'precision': 0.7790698,
 'prediction/mean': 0.3744611,
 'recall': 0.5877193,
 'global_step': 4768}

Now, let's use plain TensorFlow to implement a neural network using the tf.layers.dense class to define fully-connected (dense) layers of RELU and a softmax of the final output.

In [11]:
reset_graph()

def get_batch(X, iter, size):
    return X[(iter*batch_size) : ((iter+1)*batch_size)]



learning_rate = 0.1

num_features = X_train.shape[1]
num_instances = X_train.shape[0]

# Construction
X = tf.placeholder(tf.float32, shape=(None, num_features), name="X")
y = tf.placeholder(tf.int32, shape=(None), name="y")

with tf.name_scope("Titanic_MLP"):
    hidden1 = tf.layers.dense(X, 20, name="Hidden-1", activation = tf.nn.relu)
    hidden2 = tf.layers.dense(hidden1, 10, name="Hidden-2", activation=tf.nn.relu)
    hidden3 = tf.layers.dense(hidden2, 5, name="Hidden-3", activation=tf.nn.relu)
    logits = tf.layers.dense(hidden3, 2, name="Survived")
    
with tf.name_scope("loss"):
    xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y,
                                                              logits=logits)
    loss = tf.reduce_mean(xentropy, name="loss")
    loss_summary = tf.summary.scalar('Loss', loss)
    file_writer = tf.summary.FileWriter(logdir, tf.get_default_graph())

with tf.name_scope("train"): 
    optimizer = tf.train.GradientDescentOptimizer(learning_rate)
    training_op = optimizer.minimize(loss)
    
with tf.name_scope("eval"):
    correct = tf.nn.in_top_k(logits, y, 1)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))
    accuracy_summary = tf.summary.scalar('accuracy', accuracy)
    
init = tf.global_variables_initializer()

saver = tf.train.Saver()

n_epochs = 100
batch_size = 50


# Execution
with tf.Session() as sess:
    init.run()
    step=0
    
    for epoch in range(n_epochs):
        
        for iteration in range(num_instances // batch_size + 1):
            step+=1
            X_batch = get_batch(X_train, iteration, batch_size)
            y_batch = get_batch(y_train, iteration, batch_size)
            
            sess.run(training_op, feed_dict={X: X_batch,
                                            y: y_batch.reshape(y_batch.shape[0])})
        acc_train, loss_summary_str, acc_summary_str = sess.run([accuracy, loss_summary, accuracy_summary],feed_dict={X: X_batch, y: y_batch.reshape(y_batch.shape[0])})
        file_writer.add_summary(loss_summary_str,step)
        file_writer.add_summary(acc_summary_str,step)
        acc_val = accuracy.eval(feed_dict={X:X_test, y: y_test.reshape(y_test.shape[0])})
        
        print("{}-Train: {} Test:{}".format(epoch,
                                           acc_train,
                                           acc_val))
        
file_writer.close()

0-Train: 0.6304348111152649 Test:0.6305084824562073
1-Train: 0.695652186870575 Test:0.6542372703552246
2-Train: 0.695652186870575 Test:0.68813556432724
3-Train: 0.739130437374115 Test:0.7491525411605835
4-Train: 0.804347813129425 Test:0.8169491291046143
5-Train: 0.804347813129425 Test:0.8237287998199463
6-Train: 0.8260869383811951 Test:0.8135592937469482
7-Train: 0.8260869383811951 Test:0.8135592937469482
8-Train: 0.8478260636329651 Test:0.8169491291046143
9-Train: 0.8478260636329651 Test:0.8203389644622803
10-Train: 0.8478260636329651 Test:0.8203389644622803
11-Train: 0.8478260636329651 Test:0.8203389644622803
12-Train: 0.8695651888847351 Test:0.8203389644622803
13-Train: 0.8478260636329651 Test:0.8203389644622803
14-Train: 0.8478260636329651 Test:0.8203389644622803
15-Train: 0.8478260636329651 Test:0.8203389644622803
16-Train: 0.8478260636329651 Test:0.8203389644622803
17-Train: 0.8695651888847351 Test:0.8271186351776123
18-Train: 0.8695651888847351 Test:0.8169491291046143
19-Train: 

In [6]:
reset_graph()

encoder = OneHotEncoder(sparse=False, categories='auto')
#label_train = encoder.fit_transform(y_train)
label_test = encoder.fit_transform(y_test)


model = tf.keras.Sequential()
model.add(tf.keras.layers.Dense(20, activation='relu', input_shape=(9,)))
model.add(tf.keras.layers.Dense(10, activation='relu'))
model.add(tf.keras.layers.Dense(5, activation='relu'))
model.add(tf.keras.layers.Dense(2, activation='softmax'))

model.compile(optimizer = tf.train.GradientDescentOptimizer(0.1),loss='sparse_categorical_crossentropy',metrics=['accuracy'])

init = tf.global_variables_initializer()


# Execution
with tf.Session() as sess:
    init.run()
    model.fit(X_train, y_train, epochs=400, batch_size=100, validation_data = (X_test, y_test))

Train on 596 samples, validate on 295 samples
Epoch 1/400
596/596 [==============================] - 0s 465us/sample - loss: 0.7282 - acc: 0.4648 - val_loss: 0.6858 - val_acc: 0.6203
Epoch 2/400
596/596 [==============================] - 0s 18us/sample - loss: 0.6772 - acc: 0.6141 - val_loss: 0.6643 - val_acc: 0.6237
Epoch 3/400
596/596 [==============================] - 0s 22us/sample - loss: 0.6609 - acc: 0.6326 - val_loss: 0.6526 - val_acc: 0.6237
Epoch 4/400
596/596 [==============================] - 0s 23us/sample - loss: 0.6507 - acc: 0.6376 - val_loss: 0.6426 - val_acc: 0.6271
Epoch 5/400
596/596 [==============================] - 0s 27us/sample - loss: 0.6412 - acc: 0.6359 - val_loss: 0.6311 - val_acc: 0.6339
Epoch 6/400
596/596 [==============================] - 0s 35us/sample - loss: 0.6293 - acc: 0.6359 - val_loss: 0.6176 - val_acc: 0.6339
Epoch 7/400
596/596 [==============================] - 0s 27us/sample - loss: 0.6156 - acc: 0.6493 - val_loss: 0.6006 - val_acc: 0.6475
E

596/596 [==============================] - 0s 28us/sample - loss: 0.3985 - acc: 0.8289 - val_loss: 0.4081 - val_acc: 0.8169
Epoch 61/400
596/596 [==============================] - 0s 28us/sample - loss: 0.3989 - acc: 0.8289 - val_loss: 0.4087 - val_acc: 0.8203
Epoch 62/400
596/596 [==============================] - 0s 27us/sample - loss: 0.3977 - acc: 0.8305 - val_loss: 0.4073 - val_acc: 0.8237
Epoch 63/400
596/596 [==============================] - 0s 20us/sample - loss: 0.4007 - acc: 0.8339 - val_loss: 0.4043 - val_acc: 0.8237
Epoch 64/400
596/596 [==============================] - 0s 27us/sample - loss: 0.3997 - acc: 0.8272 - val_loss: 0.4055 - val_acc: 0.8237
Epoch 65/400
596/596 [==============================] - 0s 23us/sample - loss: 0.4012 - acc: 0.8221 - val_loss: 0.4097 - val_acc: 0.8237
Epoch 66/400
596/596 [==============================] - 0s 28us/sample - loss: 0.3976 - acc: 0.8272 - val_loss: 0.4036 - val_acc: 0.8237
Epoch 67/400
596/596 [==============================] 

596/596 [==============================] - 0s 22us/sample - loss: 0.3876 - acc: 0.8322 - val_loss: 0.4106 - val_acc: 0.8305
Epoch 120/400
596/596 [==============================] - 0s 30us/sample - loss: 0.3805 - acc: 0.8423 - val_loss: 0.4094 - val_acc: 0.8305
Epoch 121/400
596/596 [==============================] - ETA: 0s - loss: 0.4082 - acc: 0.830 - 0s 30us/sample - loss: 0.3824 - acc: 0.8372 - val_loss: 0.4122 - val_acc: 0.8203
Epoch 122/400
596/596 [==============================] - 0s 33us/sample - loss: 0.3874 - acc: 0.8440 - val_loss: 0.4106 - val_acc: 0.8271
Epoch 123/400
596/596 [==============================] - 0s 28us/sample - loss: 0.3827 - acc: 0.8406 - val_loss: 0.4139 - val_acc: 0.8271
Epoch 124/400
596/596 [==============================] - 0s 27us/sample - loss: 0.3805 - acc: 0.8406 - val_loss: 0.4100 - val_acc: 0.8271
Epoch 125/400
596/596 [==============================] - 0s 25us/sample - loss: 0.3839 - acc: 0.8423 - val_loss: 0.4110 - val_acc: 0.8271
Epoch 126/

596/596 [==============================] - 0s 25us/sample - loss: 0.3764 - acc: 0.8473 - val_loss: 0.4327 - val_acc: 0.8000
Epoch 179/400
596/596 [==============================] - 0s 32us/sample - loss: 0.3686 - acc: 0.8423 - val_loss: 0.4248 - val_acc: 0.8136
Epoch 180/400
596/596 [==============================] - 0s 28us/sample - loss: 0.3728 - acc: 0.8356 - val_loss: 0.4265 - val_acc: 0.8136
Epoch 181/400
596/596 [==============================] - ETA: 0s - loss: 0.3182 - acc: 0.890 - 0s 37us/sample - loss: 0.3720 - acc: 0.8406 - val_loss: 0.4259 - val_acc: 0.8102
Epoch 182/400
596/596 [==============================] - 0s 42us/sample - loss: 0.3687 - acc: 0.8440 - val_loss: 0.4257 - val_acc: 0.8000
Epoch 183/400
596/596 [==============================] - 0s 32us/sample - loss: 0.3758 - acc: 0.8456 - val_loss: 0.4291 - val_acc: 0.8000
Epoch 184/400
596/596 [==============================] - 0s 38us/sample - loss: 0.3691 - acc: 0.8473 - val_loss: 0.4238 - val_acc: 0.8169
Epoch 185/

596/596 [==============================] - 0s 37us/sample - loss: 0.3560 - acc: 0.8490 - val_loss: 0.4527 - val_acc: 0.8034
Epoch 238/400
596/596 [==============================] - 0s 30us/sample - loss: 0.3594 - acc: 0.8540 - val_loss: 0.4477 - val_acc: 0.7864
Epoch 239/400
596/596 [==============================] - 0s 28us/sample - loss: 0.3573 - acc: 0.8540 - val_loss: 0.4437 - val_acc: 0.8068
Epoch 240/400
596/596 [==============================] - 0s 25us/sample - loss: 0.3573 - acc: 0.8591 - val_loss: 0.4539 - val_acc: 0.7797
Epoch 241/400
596/596 [==============================] - 0s 27us/sample - loss: 0.3580 - acc: 0.8440 - val_loss: 0.4491 - val_acc: 0.7831
Epoch 242/400
596/596 [==============================] - 0s 27us/sample - loss: 0.3657 - acc: 0.8289 - val_loss: 0.4554 - val_acc: 0.7932
Epoch 243/400
596/596 [==============================] - 0s 27us/sample - loss: 0.3586 - acc: 0.8473 - val_loss: 0.4607 - val_acc: 0.7797
Epoch 244/400
596/596 [=========================

596/596 [==============================] - 0s 28us/sample - loss: 0.3525 - acc: 0.8423 - val_loss: 0.5046 - val_acc: 0.7898
Epoch 297/400
596/596 [==============================] - 0s 30us/sample - loss: 0.3523 - acc: 0.8624 - val_loss: 0.4688 - val_acc: 0.7966
Epoch 298/400
596/596 [==============================] - 0s 32us/sample - loss: 0.3426 - acc: 0.8591 - val_loss: 0.4640 - val_acc: 0.8000
Epoch 299/400
596/596 [==============================] - 0s 32us/sample - loss: 0.3419 - acc: 0.8624 - val_loss: 0.4760 - val_acc: 0.7831
Epoch 300/400
596/596 [==============================] - 0s 25us/sample - loss: 0.3441 - acc: 0.8540 - val_loss: 0.4701 - val_acc: 0.7797
Epoch 301/400
596/596 [==============================] - 0s 28us/sample - loss: 0.3484 - acc: 0.8641 - val_loss: 0.4796 - val_acc: 0.7729
Epoch 302/400
596/596 [==============================] - 0s 27us/sample - loss: 0.3580 - acc: 0.8507 - val_loss: 0.4828 - val_acc: 0.7932
Epoch 303/400
596/596 [=========================

596/596 [==============================] - 0s 27us/sample - loss: 0.3374 - acc: 0.8607 - val_loss: 0.4849 - val_acc: 0.7864
Epoch 356/400
596/596 [==============================] - 0s 28us/sample - loss: 0.3509 - acc: 0.8691 - val_loss: 0.4762 - val_acc: 0.8068
Epoch 357/400
596/596 [==============================] - 0s 30us/sample - loss: 0.3408 - acc: 0.8523 - val_loss: 0.4945 - val_acc: 0.8000
Epoch 358/400
596/596 [==============================] - 0s 33us/sample - loss: 0.3448 - acc: 0.8574 - val_loss: 0.4713 - val_acc: 0.7932
Epoch 359/400
596/596 [==============================] - ETA: 0s - loss: 0.3320 - acc: 0.870 - 0s 30us/sample - loss: 0.3353 - acc: 0.8624 - val_loss: 0.4895 - val_acc: 0.7966
Epoch 360/400
596/596 [==============================] - 0s 32us/sample - loss: 0.3361 - acc: 0.8540 - val_loss: 0.5081 - val_acc: 0.7797
Epoch 361/400
596/596 [==============================] - 0s 35us/sample - loss: 0.3379 - acc: 0.8490 - val_loss: 0.4893 - val_acc: 0.8000
Epoch 362/

# Boston Housing Cost Estimator

Building off the classifier examples above, this section shows ensemble regressors using bagging and random forests.

## Setup

In [29]:
# Load Data Set
from sklearn import datasets
boston_housing_data = datasets.load_boston()

scaler = StandardScaler()
bouston_housing_data_instances = scaler.fit_transform(boston_housing_data.data)


In [31]:
reset_graph()

train_X, test_X, train_y, test_y = train_test_split(bouston_housing_data_instances,
                                                   boston_housing_data.target,
                                                   test_size=0.33)

print(train_X.shape, train_y.shape, test_X.shape, test_y.shape)

def get_batch(X, iter, size):
    return X[(iter*batch_size) : ((iter+1)*batch_size)]

learning_rate = 0.001

num_features = train_X.shape[1]
num_instances = train_y.shape[0]

# Construction
X = tf.placeholder(tf.float32, shape=(None, num_features), name="X")
y = tf.placeholder(tf.float32, shape=(None), name="y")

with tf.name_scope("Boston-MLP"):
    hidden1 = tf.layers.dense(X, 10, name="Hidden-1", activation = tf.nn.relu)
    output = tf.layers.dense(hidden1, 1, name="Price")
    
with tf.name_scope("loss"):
    loss = tf.reduce_mean(tf.square(y-output))
    
with tf.name_scope("train"): 
    optimizer = tf.train.GradientDescentOptimizer(learning_rate)
    training_op = optimizer.minimize(loss)
    
init = tf.global_variables_initializer()
saver = tf.train.Saver()

n_epochs = 400
batch_size = 10

# Execution
with tf.Session() as sess:
    init.run()
    
    for epoch in range(n_epochs):
        
        for iteration in range(num_instances // batch_size + 1):
            X_batch = get_batch(train_X, iteration, batch_size)
            y_batch = get_batch(train_y, iteration, batch_size)
            
            sess.run(training_op, feed_dict={X: X_batch,
                                            y: y_batch})
            
        mse_train = loss.eval(feed_dict={X: X_batch,
                                            y: y_batch})
        mse_val = loss.eval(feed_dict={X:test_X, y: test_y})
        
        print("{}-Train: {} Test:{}".format(epoch,
                                           mse_train,
                                           mse_val))

(339, 13) (339,) (167, 13) (167,)
0-Train: 202.22035217285156 Test:294.7690124511719
1-Train: 89.31452941894531 Test:146.78369140625
2-Train: 77.20492553710938 Test:125.2916030883789
3-Train: 75.1042251586914 Test:119.77290344238281
4-Train: 74.22882080078125 Test:117.44296264648438
5-Train: 73.56881713867188 Test:116.1061019897461
6-Train: 73.07276916503906 Test:115.19467163085938
7-Train: 72.70021057128906 Test:114.51588439941406
8-Train: 72.38704681396484 Test:113.9810791015625
9-Train: 72.13630676269531 Test:113.55441284179688
10-Train: 71.9155044555664 Test:113.18358612060547
11-Train: 71.70333099365234 Test:112.83824920654297
12-Train: 71.49002075195312 Test:112.5195083618164
13-Train: 71.29476928710938 Test:112.23749542236328
14-Train: 71.10044860839844 Test:111.96229553222656
15-Train: 70.93395233154297 Test:111.70030975341797
16-Train: 70.753173828125 Test:111.4442367553711
17-Train: 70.631591796875 Test:111.23519897460938
18-Train: 70.54571533203125 Test:111.05506134033203
19

159-Train: 69.67720031738281 Test:108.61474609375
160-Train: 69.69322967529297 Test:108.61048889160156
161-Train: 69.69268798828125 Test:108.61351776123047
162-Train: 69.6895980834961 Test:108.61801147460938
163-Train: 69.68814849853516 Test:108.60235595703125
164-Train: 69.69866943359375 Test:108.61370086669922
165-Train: 69.6900634765625 Test:108.61006164550781
166-Train: 69.69401550292969 Test:108.61486053466797
167-Train: 69.705322265625 Test:108.5992660522461
168-Train: 69.6942367553711 Test:108.61450958251953
169-Train: 69.69859313964844 Test:108.61644744873047
170-Train: 69.7023696899414 Test:108.61133575439453
171-Train: 69.69644927978516 Test:108.60735321044922
172-Train: 69.70138549804688 Test:108.61390686035156
173-Train: 69.70396423339844 Test:108.60600280761719
174-Train: 69.6936264038086 Test:108.61036682128906
175-Train: 69.70879364013672 Test:108.60887145996094
176-Train: 69.70238494873047 Test:108.61422729492188
177-Train: 69.70930480957031 Test:108.60213470458984
178-

322-Train: 69.69769287109375 Test:109.02758026123047
323-Train: 69.70519256591797 Test:109.05591583251953
324-Train: 69.69580078125 Test:109.04165649414062
325-Train: 69.7052993774414 Test:109.0633773803711
326-Train: 69.69535827636719 Test:109.04940795898438
327-Train: 69.70071411132812 Test:109.08019256591797
328-Train: 69.71040344238281 Test:109.07901763916016
329-Train: 69.69285583496094 Test:109.0782470703125
330-Train: 69.69881439208984 Test:109.08158874511719
331-Train: 69.70426940917969 Test:109.07660675048828
332-Train: 69.70391082763672 Test:109.10965728759766
333-Train: 69.70984649658203 Test:109.09734344482422
334-Train: 69.6978759765625 Test:109.1030502319336
335-Train: 69.7125244140625 Test:109.12173461914062
336-Train: 69.69200897216797 Test:109.12173461914062
337-Train: 69.7139663696289 Test:109.12718200683594
338-Train: 69.70613861083984 Test:109.14491271972656
339-Train: 69.7054443359375 Test:109.14014434814453
340-Train: 69.70897674560547 Test:109.15225219726562
341-

From the above example, you will see that the model converges early. 

In [26]:
reset_graph()


train_X, test_X, train_y, test_y = train_test_split(bouston_housing_data_instances,
                                                   boston_housing_data.target,
                                                   test_size=0.33)


model = tf.keras.Sequential()
model.add(tf.keras.layers.Dense(10, activation='relu', input_shape=(13,)))
model.add(tf.keras.layers.Dense(1))


model.compile(optimizer = tf.train.GradientDescentOptimizer(0.001),loss='mean_squared_error',metrics=['mse'])

init = tf.global_variables_initializer()


# Execution
with tf.Session() as sess:
    init.run()
    model.fit(train_X, train_y, epochs=400, batch_size=10, validation_data = (test_X, test_y))

Train on 339 samples, validate on 167 samples


Exception ignored in: <function BaseSession._Callable.__del__ at 0x0000025CE9E0BE18>
Traceback (most recent call last):
  File "C:\ProgramData\Anaconda3\lib\site-packages\tensorflow\python\client\session.py", line 1455, in __del__
    self._session._session, self._handle, status)
  File "C:\ProgramData\Anaconda3\lib\site-packages\tensorflow\python\framework\errors_impl.py", line 528, in __exit__
    c_api.TF_GetCode(self.status.status))
tensorflow.python.framework.errors_impl.CancelledError: Session has been closed.
Exception ignored in: <function BaseSession._Callable.__del__ at 0x0000025CE9E0BE18>
Traceback (most recent call last):
  File "C:\ProgramData\Anaconda3\lib\site-packages\tensorflow\python\client\session.py", line 1455, in __del__
    self._session._session, self._handle, status)
  File "C:\ProgramData\Anaconda3\lib\site-packages\tensorflow\python\framework\errors_impl.py", line 528, in __exit__
    c_api.TF_GetCode(self.status.status))
tensorflow.python.framework.errors_im

Epoch 1/400
339/339 [==============================] - 0s 832us/sample - loss: 507.6292 - mean_squared_error: 507.6292 - val_loss: 312.0483 - val_mean_squared_error: 312.0484
Epoch 2/400
339/339 [==============================] - 0s 100us/sample - loss: 157.9132 - mean_squared_error: 157.9132 - val_loss: 71.1901 - val_mean_squared_error: 71.1901
Epoch 3/400
339/339 [==============================] - 0s 109us/sample - loss: 49.0046 - mean_squared_error: 49.0046 - val_loss: 39.3386 - val_mean_squared_error: 39.3386
Epoch 4/400
339/339 [==============================] - 0s 124us/sample - loss: 32.4291 - mean_squared_error: 32.4291 - val_loss: 30.7759 - val_mean_squared_error: 30.7759
Epoch 5/400
339/339 [==============================] - 0s 115us/sample - loss: 27.3918 - mean_squared_error: 27.3918 - val_loss: 27.5107 - val_mean_squared_error: 27.5107
Epoch 6/400
339/339 [==============================] - 0s 112us/sample - loss: 24.7225 - mean_squared_error: 24.7225 - val_loss: 24.9626 - 

Epoch 49/400
339/339 [==============================] - 0s 147us/sample - loss: 10.6789 - mean_squared_error: 10.6789 - val_loss: 14.3287 - val_mean_squared_error: 14.3287
Epoch 50/400
339/339 [==============================] - 0s 165us/sample - loss: 10.6451 - mean_squared_error: 10.6451 - val_loss: 14.2759 - val_mean_squared_error: 14.2759
Epoch 51/400
339/339 [==============================] - 0s 124us/sample - loss: 10.6512 - mean_squared_error: 10.6512 - val_loss: 13.9540 - val_mean_squared_error: 13.9540
Epoch 52/400
339/339 [==============================] - 0s 162us/sample - loss: 10.6853 - mean_squared_error: 10.6853 - val_loss: 13.9788 - val_mean_squared_error: 13.9788
Epoch 53/400
339/339 [==============================] - 0s 197us/sample - loss: 10.4073 - mean_squared_error: 10.4073 - val_loss: 14.0947 - val_mean_squared_error: 14.0947
Epoch 54/400
339/339 [==============================] - 0s 247us/sample - loss: 10.3699 - mean_squared_error: 10.3699 - val_loss: 14.5047 - 

Epoch 97/400
339/339 [==============================] - 0s 256us/sample - loss: 8.8071 - mean_squared_error: 8.8071 - val_loss: 15.4638 - val_mean_squared_error: 15.4638
Epoch 98/400
339/339 [==============================] - 0s 271us/sample - loss: 8.9782 - mean_squared_error: 8.9782 - val_loss: 14.3543 - val_mean_squared_error: 14.3543
Epoch 99/400
339/339 [==============================] - 0s 188us/sample - loss: 9.0042 - mean_squared_error: 9.0042 - val_loss: 14.6682 - val_mean_squared_error: 14.6682
Epoch 100/400
339/339 [==============================] - 0s 174us/sample - loss: 8.9110 - mean_squared_error: 8.9110 - val_loss: 14.9517 - val_mean_squared_error: 14.9516
Epoch 101/400
339/339 [==============================] - 0s 191us/sample - loss: 8.8626 - mean_squared_error: 8.8626 - val_loss: 14.8254 - val_mean_squared_error: 14.8254
Epoch 102/400
339/339 [==============================] - 0s 156us/sample - loss: 8.7286 - mean_squared_error: 8.7286 - val_loss: 15.0070 - val_mean_

...

339/339 [==============================] - 0s 174us/sample - loss: 7.0364 - mean_squared_error: 7.0364 - val_loss: 15.2249 - val_mean_squared_error: 15.2249
Epoch 193/400
339/339 [==============================] - 0s 262us/sample - loss: 7.1458 - mean_squared_error: 7.1458 - val_loss: 15.3979 - val_mean_squared_error: 15.3979
Epoch 194/400
339/339 [==============================] - 0s 221us/sample - loss: 6.9578 - mean_squared_error: 6.9578 - val_loss: 15.4545 - val_mean_squared_error: 15.4545
Epoch 195/400
339/339 [==============================] - 0s 185us/sample - loss: 7.0647 - mean_squared_error: 7.0647 - val_loss: 15.0985 - val_mean_squared_error: 15.0985
Epoch 196/400
339/339 [==============================] - 0s 118us/sample - loss: 7.0419 - mean_squared_error: 7.0419 - val_loss: 15.9515 - val_mean_squared_error: 15.9515
Epoch 197/400
339/339 [==============================] - 0s 144us/sample - loss: 6.9213 - mean_squared_error: 6.9213 - val_loss: 15.8552 - val_mean_squared_err

Epoch 240/400
339/339 [==============================] - 0s 247us/sample - loss: 6.4864 - mean_squared_error: 6.4864 - val_loss: 16.3365 - val_mean_squared_error: 16.3365
Epoch 241/400
339/339 [==============================] - 0s 294us/sample - loss: 6.5665 - mean_squared_error: 6.5665 - val_loss: 16.6527 - val_mean_squared_error: 16.6527
Epoch 242/400
339/339 [==============================] - 0s 159us/sample - loss: 6.5303 - mean_squared_error: 6.5303 - val_loss: 16.4519 - val_mean_squared_error: 16.4519
Epoch 243/400
339/339 [==============================] - 0s 185us/sample - loss: 6.4707 - mean_squared_error: 6.4707 - val_loss: 16.0724 - val_mean_squared_error: 16.0724
Epoch 244/400
339/339 [==============================] - 0s 147us/sample - loss: 6.3673 - mean_squared_error: 6.3673 - val_loss: 15.5737 - val_mean_squared_error: 15.5737
Epoch 245/400
339/339 [==============================] - 0s 141us/sample - loss: 6.4870 - mean_squared_error: 6.4870 - val_loss: 15.5661 - val_me

Epoch 288/400
339/339 [==============================] - 0s 162us/sample - loss: 6.1723 - mean_squared_error: 6.1723 - val_loss: 15.9058 - val_mean_squared_error: 15.9058
Epoch 289/400
339/339 [==============================] - 0s 159us/sample - loss: 6.1109 - mean_squared_error: 6.1109 - val_loss: 15.6573 - val_mean_squared_error: 15.6573
Epoch 290/400
339/339 [==============================] - 0s 215us/sample - loss: 6.1362 - mean_squared_error: 6.1362 - val_loss: 16.2043 - val_mean_squared_error: 16.2043
Epoch 291/400
339/339 [==============================] - 0s 162us/sample - loss: 6.0889 - mean_squared_error: 6.0889 - val_loss: 15.2511 - val_mean_squared_error: 15.2511
Epoch 292/400
339/339 [==============================] - 0s 121us/sample - loss: 6.0848 - mean_squared_error: 6.0848 - val_loss: 15.7082 - val_mean_squared_error: 15.7082
Epoch 293/400
339/339 [==============================] - 0s 135us/sample - loss: 6.0713 - mean_squared_error: 6.0713 - val_loss: 15.9258 - val_me

339/339 [==============================] - 0s 294us/sample - loss: 5.7967 - mean_squared_error: 5.7967 - val_loss: 15.5218 - val_mean_squared_error: 15.5218
Epoch 336/400
339/339 [==============================] - 0s 168us/sample - loss: 5.8567 - mean_squared_error: 5.8567 - val_loss: 15.8726 - val_mean_squared_error: 15.8726
Epoch 337/400
339/339 [==============================] - 0s 247us/sample - loss: 5.8752 - mean_squared_error: 5.8752 - val_loss: 15.5354 - val_mean_squared_error: 15.5354
Epoch 338/400
339/339 [==============================] - 0s 162us/sample - loss: 5.8293 - mean_squared_error: 5.8293 - val_loss: 15.8133 - val_mean_squared_error: 15.8133
Epoch 339/400
339/339 [==============================] - 0s 179us/sample - loss: 5.7939 - mean_squared_error: 5.7939 - val_loss: 15.6419 - val_mean_squared_error: 15.6419
Epoch 340/400
339/339 [==============================] - 0s 171us/sample - loss: 5.7278 - mean_squared_error: 5.7278 - val_loss: 15.2032 - val_mean_squared_err

Epoch 383/400
339/339 [==============================] - 0s 144us/sample - loss: 5.4960 - mean_squared_error: 5.4960 - val_loss: 16.2311 - val_mean_squared_error: 16.2311
Epoch 384/400
339/339 [==============================] - 0s 144us/sample - loss: 5.6197 - mean_squared_error: 5.6197 - val_loss: 15.4801 - val_mean_squared_error: 15.4801
Epoch 385/400
339/339 [==============================] - 0s 127us/sample - loss: 5.6154 - mean_squared_error: 5.6154 - val_loss: 15.1186 - val_mean_squared_error: 15.1186
Epoch 386/400
339/339 [==============================] - 0s 91us/sample - loss: 5.5802 - mean_squared_error: 5.5802 - val_loss: 14.6211 - val_mean_squared_error: 14.6211
Epoch 387/400
339/339 [==============================] - 0s 94us/sample - loss: 5.6793 - mean_squared_error: 5.6793 - val_loss: 15.0756 - val_mean_squared_error: 15.0756
Epoch 388/400
339/339 [==============================] - 0s 100us/sample - loss: 5.6295 - mean_squared_error: 5.6295 - val_loss: 15.4963 - val_mean